# Investigating relationships between gene-expression and anatomy/connectivity in Gilles de la Tourette syndrome

* This notebook illustrates the strategy implemented to construct the overall Allen Human Brain Atlas dataframe.
* The dataframe consists of the microarray expression values of 20,737 genes extracted from 3702 tissue samples (wells) from 6 healty donors (post-mortem).


* Donor data includes:
    * H0351.2001, 24 yrs, M, Black or African American
    * H0351.2002, 39 yrs, M, Black or African American
    * H0351.1009, 57 yrs, M, White or Caucasian
    * H0351.1012, 31 yrs, M, White or Caucasian
    * H0351.1015, 49 yrs, F, Hispanic
    * H0351.1016, 55 yrs, M, White or Caucasian
    * Data can be downloaded from the following URL: http://human.brain-map.org/static/download
    
    
* Metadata included in the constructed dataframe includes:
    * MRI coordiantes of tissue samples locations in MNI standard space.
    * Structure names, ids, and relative parent structure names for each of the 3702 wells.
    * Detailed ontology of each brain structure for each well.
    
    
* The metadata is initually used to map each well to the closests ROI in a specific parcellation.
* Subsequently, gene expression values are mapped across the cortex based on this parcellation (See notebook 001)


* Related Papers Include: 
    * **Whitaker, PNAS, 2016**: Adolescence is associated with genomically patterned consolidation of the hubs of the human brain connectome.*     
    **Romme, Biological Psychiatry, 2017**: Connectome Disconnectivity and Cortical Gene Expression in Patients With Schizophrenia
    * **Wong, Cerebral Cortex  2017**: Inter-Regional Variations in Gene Expression and Age-Related Cortical Thinning in the Adolescent Brain
    * **Shin, Cerebral Cortex, 2016**: Cell-Specific Gene-Expression Profiles and Cortical Thickness in the Human Brain
    * **Romero-Garcia, OHBM, 2017**: Gene expression analysis of structural cortical differences in autism
    * **Richiardi, Science, 2015**: Correlated gene expression supports synchronous activity in brain networks




In [1]:
import os 
import zipfile
import StringIO
import pandas as pd
pd.options.display.max_rows= 1000
pd.options.display.max_columns= 30

datadir = '/scr/malta1/Github/ENIGMA_TOURETTE/AHBA/ahba_data'
donors = ['donor1', 'donor2', 'donor3', 'donor4', 'donor5', 'donor6']

=========================
Allen Human Brain Atlas
=========================

This data set contains all microarray samples for all probes in 1 brain.

MicroarrayExpression.csv

    Contains normalized expression values arranged by (row, column) =
    (probe, samples).  The first column is the probe ID.

PACall.csv 

    Contains a present/absent flag which indicates whether the probe's
    expression is well above background.  It is set to 1 when both of the
    following conditions are met.
    
        1) The 2-sided t-test p-value is lower than 0.01, (indicating the mean
           signal of the probe's expression is significantly different from the
           corresponding background).
        2) The difference between the background subtracted signal and the
           background is significant (> 2.6 * background standard deviation).

Probes.csv

    Metadata for the probes in MicroarrayExpression.csv.

Ontology.csv

    The ontology on brain structures used for sampling.

SampleAnnot.csv

    The samples are listed in the same order as the columns in
    MicroarrayExpression.csv; the locations of all sampled structures are given
    in native MRI voxel coordinates (mri_cx, mri_cy, mri_cz) as well as MNI
    coordinates.

The MRI data can be downloaded from http://human.brain-map.org/mri_viewers/data.

In [2]:
def return_donor_gene_expression(donor):
    
    #I/O
    donor_zip  = zipfile.ZipFile(os.path.join(datadir,'%s.zip'%donor))
    
    #read microarray data. take care indexing.... should have 58692 probes
    df_microarray = pd.read_csv(StringIO.StringIO(donor_zip.read('MicroarrayExpression.csv')),
                                header= None,index_col =0 )
    
    # read probe meta-data
    df_probes = pd.read_csv(StringIO.StringIO(donor_zip.read('Probes.csv')), index_col =0 )
    del df_probes.index.name
    
    # drop unannotated genes... should end up with 48170 probes
    drop_strings = ['uncharacterized', 'non-RefSeq']
    drop_strings = ['non-RefSeq']
    drop_probes  = [i for i in df_probes.index for x in drop_strings if x in df_probes.loc[i]['gene_name'] ]
    df_probes = df_probes.drop(drop_probes, axis =0)
    
    # Calc mean probe expression for each gene
    df_gene_exp = pd.concat([df_microarray, df_probes['gene_symbol']],axis=1).groupby('gene_symbol').mean().T.reset_index().drop(['index'],axis=1)
    df_gene_exp.columns.name = None
    
    # Add metadata to dataframe
    df_meta = pd.read_csv(StringIO.StringIO(donor_zip.read('SampleAnnot.csv')))
    df = pd.concat([df_gene_exp,df_meta],axis=1).set_index('well_id')
    del df.index.name
    
    # Append donor name
    df['donor'] = donor
    
    # Append ontology of brain structures used for sampling
    df_ontology = pd.read_csv(StringIO.StringIO(donor_zip.read('Ontology.csv')), index_col =0 )
    df_ontology_dict = {}
    for i in df_ontology.index:
        df_ontology_dict[i] = df_ontology.loc[i]['name']

    for well in df.index:
        struct_id = df.loc[well]['structure_id']
        df.loc[well,'parent_structure_id'] = df_ontology.loc[struct_id]['color_hex_triplet']
        df.loc[well,'hemisphere'] = df_ontology.loc[struct_id]['hemisphere']
        df.loc[well,'graph_order'] = df_ontology.loc[struct_id]['graph_order']
        df.loc[well,'structure_id_path'] = df_ontology.loc[struct_id]['structure_id_path']
        df.loc[well,'color_hex_triplet'] = df_ontology.loc[struct_id]['color_hex_triplet']
        
        # rename structure_id_path to specific anatomical ontology
        ontology = filter(None, [i for i in  df.loc[well,'structure_id_path'].split("/")])
        ontology = [df_ontology_dict[int(i)] for i in ontology]
        df.loc[well,'Ontology'] = '//'.join(ontology)
        
        # Append new columns with all ontologies
        for i,name in enumerate(ontology):
            df.loc[well,'Ontology_%s'%i] = name
    
    return df


In [3]:
df_all = []
for donor in donors:
    df = return_donor_gene_expression(donor)
    df_all.append(df)

In [4]:
df_all[0].head()

,61E3.4,A1BG,A1CF,A26C1B,A2BP1,A2LD1,A2M,A2ML1,A3GALT2P,A4GALT,A4GNT,AAAS,AACS,AADAC,AADACL1,...,parent_structure_id,hemisphere,graph_order,structure_id_path,color_hex_triplet,Ontology,Ontology_0,Ontology_1,Ontology_2,Ontology_3,Ontology_4,Ontology_5,Ontology_6,Ontology_7,Ontology_8
594,12.257452,4.307083,1.639045,6.047821,11.502535,2.992096,7.702302,4.657058,1.675294,3.470479,1.694466,5.580722,5.942118,2.430642,8.623031,...,E8CF59,R,60.0,/4005/4006/4007/4008/4009/4071/4075/4077/,E8CF59,brain//gray matter//telencephalon//cerebral co...,brain,gray matter,telencephalon,cerebral cortex,frontal lobe,"paracentral lobule, anterior part","paracentral lobule, anterior part, right","paracentral lobule, anterior part, right, infe...",NaN
2985,12.971966,5.394092,2.016619,6.258501,12.314500,3.450627,8.285363,5.044534,3.596557,4.651128,1.979790,5.187786,4.888155,1.860784,9.476809,...,C0A7FF,R,504.0,/4005/4006/4007/4275/4321/4323/,C0A7FF,brain//gray matter//telencephalon//cerebral nu...,brain,gray matter,telencephalon,cerebral nuclei,claustrum,"claustrum, right",NaN,NaN,NaN
2801,13.397286,6.196477,2.419608,5.442959,12.776440,3.652065,8.698773,4.777458,3.662503,4.845015,2.339243,5.557337,4.499551,2.292158,9.850784,...,C0A7FF,R,504.0,/4005/4006/4007/4275/4321/4323/,C0A7FF,brain//gray matter//telencephalon//cerebral nu...,brain,gray matter,telencephalon,cerebral nuclei,claustrum,"claustrum, right",NaN,NaN,NaN
2273,13.326607,5.665946,2.473735,4.363560,12.428296,4.512593,9.333068,5.325190,2.956374,3.377773,2.303905,4.249260,3.528285,2.275111,9.354156,...,A9E855,L,726.0,/4005/4006/4391/4392/4393/12922/4440/,A9E855,brain//gray matter//diencephalon//thalamus//do...,brain,gray matter,diencephalon,thalamus,dorsal thalamus,dorsal lateral geneiculate nucleus,"dorsal lateral geniculate nucleus, left",NaN,NaN
2785,13.356659,5.087975,2.391748,4.590116,13.365749,3.558010,9.939992,5.247008,3.760852,4.446860,2.307497,5.293467,3.392771,2.185038,9.783472,...,FFC966,R,139.0,/4005/4006/4007/4008/4219/4249/12895/4266/,FFC966,brain//gray matter//telencephalon//cerebral co...,brain,gray matter,telencephalon,cerebral cortex,limbic lobe,hippocampal formation,CA4 field,"CA4 field, right",NaN


In [5]:
df_all[1].head()

,61E3.4,A1BG,A1CF,A26C1B,A2BP1,A2LD1,A2M,A2ML1,A3GALT2P,A4GALT,A4GNT,AAAS,AACS,AADAC,AADACL1,...,parent_structure_id,hemisphere,graph_order,structure_id_path,color_hex_triplet,Ontology,Ontology_0,Ontology_1,Ontology_2,Ontology_3,Ontology_4,Ontology_5,Ontology_6,Ontology_7,Ontology_8
5705,12.233134,5.321921,2.166305,5.482180,11.819855,2.851921,8.042239,5.062174,2.075208,3.414446,2.094129,5.160365,4.870299,2.076208,9.144500,...,C0A7FF,L,503.0,/4005/4006/4007/4275/4321/4322/,C0A7FF,brain//gray matter//telencephalon//cerebral nu...,brain,gray matter,telencephalon,cerebral nuclei,claustrum,"claustrum, left",NaN,NaN,NaN
5713,12.316856,5.018191,1.961696,6.134849,12.436989,3.642371,8.557210,5.018321,1.279049,3.050612,1.340674,5.730010,4.263612,1.409871,9.355832,...,895BFF,L,497.0,/4005/4006/4007/4275/4276/4277/4290/4291/,895BFF,brain//gray matter//telencephalon//cerebral nu...,brain,gray matter,telencephalon,cerebral nuclei,basal ganglia,striatum,nucleus accumbens,"nucleus accumbens, left",NaN
5721,12.374035,5.790579,1.333967,7.122039,12.883375,4.371786,9.890256,4.266128,2.316213,2.814452,1.737654,6.193731,4.774909,1.392683,10.441353,...,895BFF,R,498.0,/4005/4006/4007/4275/4276/4277/4290/4292/,895BFF,brain//gray matter//telencephalon//cerebral nu...,brain,gray matter,telencephalon,cerebral nuclei,basal ganglia,striatum,nucleus accumbens,"nucleus accumbens, right",NaN
5729,12.168797,5.152017,1.279049,5.969047,11.867710,3.600621,8.783679,4.262765,1.869106,3.151701,1.279049,5.514186,4.622687,1.279049,9.046109,...,895BFF,R,498.0,/4005/4006/4007/4275/4276/4277/4290/4292/,895BFF,brain//gray matter//telencephalon//cerebral nu...,brain,gray matter,telencephalon,cerebral nuclei,basal ganglia,striatum,nucleus accumbens,"nucleus accumbens, right",NaN
5737,11.663920,5.442288,1.618193,5.381421,11.892481,3.462606,8.473183,4.514223,1.279049,2.878600,1.843032,4.832677,4.172980,1.731081,9.306796,...,D168FF,R,468.0,/4005/4006/4007/4275/4300/13003/4314/,D168FF,brain//gray matter//telencephalon//cerebral nu...,brain,gray matter,telencephalon,cerebral nuclei,basal forebrain,substantia innominata,"substantia innominata, right",NaN,NaN


In [6]:
df_all[2].head()

,61E3.4,A1BG,A1CF,A26C1B,A2BP1,A2LD1,A2M,A2ML1,A3GALT2P,A4GALT,A4GNT,AAAS,AACS,AADAC,AADACL1,...,parent_structure_id,hemisphere,graph_order,structure_id_path,color_hex_triplet,Ontology,Ontology_0,Ontology_1,Ontology_2,Ontology_3,Ontology_4,Ontology_5,Ontology_6,Ontology_7,Ontology_8
11281,12.889573,5.049439,2.042317,6.168989,12.040668,3.063961,8.695483,5.388024,3.393456,3.140641,1.900542,6.609978,6.347626,1.817107,8.892194,...,FF907F,L,282.0,/4005/4006/4007/4008/4132/4140/4141/4143/,FF907F,brain//gray matter//telencephalon//cerebral co...,brain,gray matter,telencephalon,cerebral cortex,temporal lobe,middle temporal gyrus,"middle temporal gyrus, left","middle temporal gyrus, left, inferior bank of ...",NaN
11305,13.003047,5.173558,1.973284,6.154843,12.183389,2.988897,8.257742,6.092457,3.255138,2.862612,1.948346,6.165471,5.699771,2.633926,9.048076,...,FF8D7F,L,274.0,/4005/4006/4007/4008/4132/4147/4148/4151/,FF8D7F,brain//gray matter//telencephalon//cerebral co...,brain,gray matter,telencephalon,cerebral cortex,temporal lobe,inferior temporal gyrus,"inferior temporal gyrus, left","inferior temporal gyrus, left, bank of mts",NaN
11289,12.998348,5.068065,1.996434,6.158318,12.233327,3.140788,8.227760,5.474782,2.792448,3.270367,2.105640,5.941537,6.258556,2.034660,9.060142,...,FF8D7F,L,272.0,/4005/4006/4007/4008/4132/4147/4148/4149/,FF8D7F,brain//gray matter//telencephalon//cerebral co...,brain,gray matter,telencephalon,cerebral cortex,temporal lobe,inferior temporal gyrus,"inferior temporal gyrus, left","inferior temporal gyrus, left, bank of the its",NaN
11335,13.128374,5.114931,1.981823,6.334744,12.369833,2.990184,8.547374,5.556616,2.997993,3.002738,2.020319,6.370282,5.788600,1.999360,9.178984,...,FF907F,L,281.0,/4005/4006/4007/4008/4132/4140/4141/4142/,FF907F,brain//gray matter//telencephalon//cerebral co...,brain,gray matter,telencephalon,cerebral cortex,temporal lobe,middle temporal gyrus,"middle temporal gyrus, left","middle temporal gyrus, left, superior bank of ...",NaN
11319,12.633394,4.945027,2.108420,6.269155,12.110327,2.843943,7.657725,5.173323,1.969178,2.934619,2.252099,6.037815,5.947000,2.164782,8.976726,...,FF997F,L,294.0,/4005/4006/4007/4008/4132/4133/4134/4135/,FF997F,brain//gray matter//telencephalon//cerebral co...,brain,gray matter,telencephalon,cerebral cortex,temporal lobe,superior temporal gyrus,"superior temporal gyrus, left","superior temporal gyrus, left, lateral bank of...",NaN


In [7]:
df_all[3].head()

,61E3.4,A1BG,A1CF,A26C1B,A2BP1,A2LD1,A2M,A2ML1,A3GALT2P,A4GALT,A4GNT,AAAS,AACS,AADAC,AADACL1,...,parent_structure_id,hemisphere,graph_order,structure_id_path,color_hex_triplet,Ontology,Ontology_0,Ontology_1,Ontology_2,Ontology_3,Ontology_4,Ontology_5,Ontology_6,Ontology_7,Ontology_8
126439333,12.668165,5.265878,1.629761,5.284850,9.320815,3.434027,9.591122,4.672892,4.361163,3.198709,1.605798,6.797188,4.607100,1.541086,8.572915,...,9C53E8,L,483.0,/4005/4006/4007/4275/4276/4293/12898/4296/,9C53E8,brain//gray matter//telencephalon//cerebral nu...,brain,gray matter,telencephalon,cerebral nuclei,basal ganglia,globus pallidus,"globus pallidus, internal segment","globus pallidus, internal segment, left",NaN
126432538,12.383563,4.984746,1.902150,6.602719,11.705152,3.027106,8.300779,4.594322,3.905210,3.510353,1.481738,6.433695,6.212185,1.472261,8.846980,...,FF8A7F,L,268.0,/4005/4006/4007/4008/4132/4165/4166/,FF8A7F,brain//gray matter//telencephalon//cerebral co...,brain,gray matter,telencephalon,cerebral cortex,temporal lobe,Heschl's gyrus,"Heschl's gyrus, left",NaN,NaN
126435762,12.433869,5.256781,2.841570,6.023792,10.619858,2.962560,8.668816,4.849864,1.242969,2.848064,1.587719,6.370300,4.373975,1.746981,8.402952,...,00D5E8,L,1044.0,/4005/4006/4833/4696/4697/12930/12931/12934/4735/,00D5E8,brain//gray matter//metencephalon//cerebellum/...,brain,gray matter,metencephalon,cerebellum,cerebellar cortex,cerebellar hemispheres,Anterior Lobe,IV,"IV, left, lateral hemisphere"
126439285,12.604221,6.025966,1.878753,5.825703,11.514246,3.229238,8.925490,5.839518,3.116854,3.311976,1.492444,6.074064,5.917818,1.516755,8.716836,...,BF934C,L,156.0,/4005/4006/4007/4008/4219/4242/4243/4244/,BF934C,brain//gray matter//telencephalon//cerebral co...,brain,gray matter,telencephalon,cerebral cortex,limbic lobe,parahippocampal gyrus,"parahippocampal gyrus, left","parahippocampal gyrus, left, lateral bank of g...",NaN
126435744,12.397088,5.580429,1.922846,6.643124,11.544962,3.647850,8.861175,4.964262,3.780269,2.258278,2.827696,6.804749,5.606154,1.459874,8.823481,...,BF934C,L,156.0,/4005/4006/4007/4008/4219/4242/4243/4244/,BF934C,brain//gray matter//telencephalon//cerebral co...,brain,gray matter,telencephalon,cerebral cortex,limbic lobe,parahippocampal gyrus,"parahippocampal gyrus, left","parahippocampal gyrus, left, lateral bank of g...",NaN


In [8]:
df_all[4].head()

,61E3.4,A1BG,A1CF,A26C1B,A2BP1,A2LD1,A2M,A2ML1,A3GALT2P,A4GALT,A4GNT,AAAS,AACS,AADAC,AADACL1,...,parent_structure_id,hemisphere,graph_order,structure_id_path,color_hex_triplet,Ontology,Ontology_0,Ontology_1,Ontology_2,Ontology_3,Ontology_4,Ontology_5,Ontology_6,Ontology_7,Ontology_8
141667089,12.216923,4.878327,1.351646,6.541192,12.182163,2.458788,7.840581,5.184563,3.081331,3.899325,1.435471,5.870518,6.324569,1.407365,9.244058,...,E88F51,L,205.0,/4005/4006/4007/4008/4180/4212/4213/4215/,E88F51,brain//gray matter//telencephalon//cerebral co...,brain,gray matter,telencephalon,cerebral cortex,occipital lobe,superior occipital gyrus,"superior occipital gyrus, left","superior occipital gyrus, left, inferior bank ...",NaN
141667177,12.415516,5.248741,1.700870,6.462244,11.794496,2.519380,7.854143,5.331488,2.190917,3.530119,1.243976,5.994728,6.689680,1.257583,8.981299,...,7F6933,L,116.0,/4005/4006/4007/4008/4219/4220/4235/4236/4237/,7F6933,brain//gray matter//telencephalon//cerebral co...,brain,gray matter,telencephalon,cerebral cortex,limbic lobe,cingulate gyrus,"cingulate gyrus, retrosplenial part","cingulate gyrus, retrosplenial part, left","cingulate gyrus, retrosplenial part, left, sup..."
141667207,12.578205,5.079578,1.431586,6.579631,12.365258,2.414339,8.479937,5.179049,3.183998,3.481347,1.349340,5.965350,6.510166,1.366529,9.122224,...,FF6C62,L,237.0,/4005/4006/4007/4008/4084/4085/4086/4090/,FF6C62,brain//gray matter//telencephalon//cerebral co...,brain,gray matter,telencephalon,cerebral cortex,parietal lobe,postcentral gyrus,"postcentral gyrus, left","postcentral gyrus, left, bank of the posterior...",NaN
139409861,12.566850,5.368312,1.351722,6.174279,12.179179,2.896632,8.494656,5.720428,2.664837,4.214436,1.472095,5.248285,6.027942,1.485595,8.933423,...,FF5D62,L,221.0,/4005/4006/4007/4008/4084/4103/4104/4105/4107/,FF5D62,brain//gray matter//telencephalon//cerebral co...,brain,gray matter,telencephalon,cerebral cortex,parietal lobe,inferior parietal lobule,supramarginal gyrus,"supramarginal gyrus, left","supramarginal gyrus, left, inferior bank of gyrus"
139409845,12.524196,5.294441,2.305288,6.291404,12.227917,1.961407,8.246789,5.077249,3.165482,3.797439,1.187811,6.000128,6.760649,1.209626,9.031611,...,FF5D62,L,220.0,/4005/4006/4007/4008/4084/4103/4104/4105/4106/,FF5D62,brain//gray matter//telencephalon//cerebral co...,brain,gray matter,telencephalon,cerebral cortex,parietal lobe,inferior parietal lobule,supramarginal gyrus,"supramarginal gyrus, left","supramarginal gyrus, left, superior bank of gyrus"


In [9]:
df_all[5].head()

,61E3.4,A1BG,A1CF,A26C1B,A2BP1,A2LD1,A2M,A2ML1,A3GALT2P,A4GALT,A4GNT,AAAS,AACS,AADAC,AADACL1,...,parent_structure_id,hemisphere,graph_order,structure_id_path,color_hex_triplet,Ontology,Ontology_0,Ontology_1,Ontology_2,Ontology_3,Ontology_4,Ontology_5,Ontology_6,Ontology_7,Ontology_8
160535133,11.910527,4.393986,0.974908,5.037044,11.441434,3.049757,8.951266,4.993544,3.516486,2.603628,1.035412,5.826224,4.333930,1.647601,9.703043,...,00FFAA,L,1135.0,/4005/4006/4833/9131/9135/9142/9143/,00FFAA,brain//gray matter//metencephalon//pons//ponti...,brain,gray matter,metencephalon,pons,pontine tegmentum,facial motor nucleus,"facial motor nucleus, left",NaN,NaN
160535307,12.404726,4.289177,1.028869,4.684117,10.300250,2.804438,9.614341,5.126696,3.708341,3.289085,2.536449,5.526525,4.310811,1.105992,9.558277,...,00FFAA,L,1114.0,/4005/4006/4833/9131/9135/9136/9137/,00FFAA,brain//gray matter//metencephalon//pons//ponti...,brain,gray matter,metencephalon,pons,pontine tegmentum,abducens nucleus,"abducens nucleus, left",NaN,NaN
160535291,12.567677,5.225004,1.340397,5.308471,10.072753,3.232524,9.427397,4.835249,5.282029,3.722703,1.302413,6.176882,4.258769,1.865744,8.066712,...,00FFAA,L,1171.0,/4005/4006/4833/9131/9135/9157/9158/,00FFAA,brain//gray matter//metencephalon//pons//ponti...,brain,gray matter,metencephalon,pons,pontine tegmentum,pontine raphe nucleus,"pontine raphe nucleus, left",NaN,NaN
160535275,13.013984,5.496016,1.206306,6.177618,10.144302,3.361947,10.197965,5.380921,4.737649,3.909283,1.157304,6.302501,4.549600,1.163711,9.391232,...,3D85EA,L,1415.0,/4005/4006/9512/9697/9698/,3D85EA,brain//gray matter//myelencephalon//vestibular...,brain,gray matter,myelencephalon,vestibular nuclei,"vestibular nuclei, left",NaN,NaN,NaN,NaN
160535259,12.574042,5.412059,1.547726,5.949662,10.814045,3.312976,8.985808,5.082649,4.651419,3.129015,1.045292,5.805359,5.381162,0.986515,9.300912,...,377FE4,L,1394.0,/4005/4006/9512/9676/9677/,377FE4,brain//gray matter//myelencephalon//spinal tri...,brain,gray matter,myelencephalon,spinal trigeminal nucleus,"spinal trigeminal nucleus, left",NaN,NaN,NaN,NaN


In [10]:
print 'Number of Tissue samples = ',946+893+363+529+470+501,'wells'

Number of Tissue samples =  3702 wells


In [11]:
AHBA = pd.concat(df_all, axis = 0)

In [12]:
AHBA.to_csv(os.path.join(datadir,'AHBA_20737.csv'))

In [13]:
AHBA.iloc[:,-28:].head()

,structure_id,slab_num,slab_type,structure_acronym,structure_name,polygon_id,mri_voxel_x,mri_voxel_y,mri_voxel_z,mni_x,mni_y,mni_z,donor,parent_structure_id,hemisphere,graph_order,structure_id_path,color_hex_triplet,Ontology,Ontology_0,Ontology_1,Ontology_2,Ontology_3,Ontology_4,Ontology_5,Ontology_6,Ontology_7,Ontology_8
594,4077,22,CX,PCLa-i,"paracentral lobule, anterior part, right, infe...",37470,87,52,116,5.9,-27.7,49.7,donor1,E8CF59,R,60.0,/4005/4006/4007/4008/4009/4071/4075/4077/,E8CF59,brain//gray matter//telencephalon//cerebral co...,brain,gray matter,telencephalon,cerebral cortex,frontal lobe,"paracentral lobule, anterior part","paracentral lobule, anterior part, right","paracentral lobule, anterior part, right, infe...",NaN
2985,4323,11,CX,Cl,"claustrum, right",40517,66,92,63,29.2,17.0,-2.9,donor1,C0A7FF,R,504.0,/4005/4006/4007/4275/4321/4323/,C0A7FF,brain//gray matter//telencephalon//cerebral nu...,brain,gray matter,telencephalon,cerebral nuclei,claustrum,"claustrum, right",NaN,NaN,NaN
2801,4323,18,CX,Cl,"claustrum, right",41516,66,81,104,28.2,-22.8,16.8,donor1,C0A7FF,R,504.0,/4005/4006/4007/4275/4321/4323/,C0A7FF,brain//gray matter//telencephalon//cerebral nu...,brain,gray matter,telencephalon,cerebral nuclei,claustrum,"claustrum, right",NaN,NaN,NaN
2273,4440,18,CX,LGd,"dorsal lateral geniculate nucleus, left",41473,116,94,101,-24.6,-24.6,1.3,donor1,A9E855,L,726.0,/4005/4006/4391/4392/4393/12922/4440/,A9E855,brain//gray matter//diencephalon//thalamus//do...,brain,gray matter,diencephalon,thalamus,dorsal thalamus,dorsal lateral geneiculate nucleus,"dorsal lateral geniculate nucleus, left",NaN,NaN
2785,4266,17,CX,CA4,"CA4 field, right",41142,63,104,106,31.1,-31.3,-7.3,donor1,FFC966,R,139.0,/4005/4006/4007/4008/4219/4249/12895/4266/,FFC966,brain//gray matter//telencephalon//cerebral co...,brain,gray matter,telencephalon,cerebral cortex,limbic lobe,hippocampal formation,CA4 field,"CA4 field, right",NaN


In [14]:
ontology3 = set([AHBA.loc[well]['Ontology_3'] for well in AHBA.index])
ontology3

{'arcuate nucleus of medulla',
 'central glial substance',
 'cerebellum',
 'cerebral cortex',
 'cerebral nuclei',
 'cochlear nuclei',
 'cuneate nucleus',
 'dorsal motor nucleus of the vagus',
 'epithalamus',
 'gracile nucleus',
 'hypoglossal nucleus',
 'hypothalamus',
 'inferior olivary complex',
 'lateral ventricle',
 'medullary reticular formation',
 'midbrain tectum',
 'midbrain tegmentum',
 'pons',
 'pretectal region',
 'raphe nuclei of medulla',
 'spinal trigeminal nucleus',
 'subthalamus',
 'telencephalic commissures',
 'telencephalic white matter tracts',
 'thalamus',
 'vestibular nuclei'}

In [15]:
'test'

'test'

In [16]:
'test'

'test'